# Сonversion functions

In [4]:
CREATE OR REPLACE FUNCTION fn_word_to_bigint(_word TEXT)
RETURNS BIGINT
IMMUTABLE PARALLEL SAFE
AS
$$

SELECT  ASCII(SUBSTRING(_word FROM 1 FOR 1))::BIGINT +
        (ASCII(SUBSTRING(_word FROM 2 FOR 1))::BIGINT << 8) +
        (ASCII(SUBSTRING(_word FROM 3 FOR 1))::BIGINT << 16) +
        (ASCII(SUBSTRING(_word FROM 4 FOR 1))::BIGINT << 24) +
        (ASCII(SUBSTRING(_word FROM 5 FOR 1))::BIGINT << 32)

$$
LANGUAGE 'sql';


CREATE OR REPLACE FUNCTION fn_bigint_to_word(_digits BIGINT)
RETURNS TEXT
IMMUTABLE PARALLEL SAFE
AS
$$

SELECT  CHR((_digits & 255)::INT) ||
        CHR(((_digits >> 8) & 255)::INT) ||
        CHR(((_digits >> 16) & 255)::INT) ||
        CHR(((_digits >> 24) & 255)::INT) ||
        CHR(((_digits >> 32) & 255)::INT)

$$
LANGUAGE 'sql';

Commands completed successfully

Commands completed successfully

Общее время выполнения: 00:00:00.087

# Match functions

In [5]:
CREATE OR REPLACE FUNCTION fn_match(_guess BIGINT, _target BIGINT, _position INT)
RETURNS INT
IMMUTABLE PARALLEL SAFE
COST 10000
AS

$$

SELECT  CASE
        WHEN ((_guess >> (8 * _position)) & 255) = ((_target >> (8 * _position)) & 255) THEN 2
        ELSE    (
                (((_guess >> (8 * _position)) & 255) = ((_target >> 0) & 255)
                        AND ((_guess >> 0) & 255) <> ((_target >> 0) & 255)
                        )::INT +
                (((_guess >> (8 * _position)) & 255) = ((_target >> 8) & 255)
                        AND ((_guess >> 8) & 255) <> ((_target >> 8) & 255)
                        )::INT +
                (((_guess >> (8 * _position)) & 255) = ((_target >> 16) & 255)
                        AND ((_guess >> 16) & 255) <> ((_target >> 16) & 255)
                        )::INT +
                (((_guess >> (8 * _position)) & 255) = ((_target >> 24) & 255)
                        AND ((_guess >> 24) & 255) <> ((_target >> 24) & 255)
                        )::INT +
                (((_guess >> (8 * _position)) & 255) = ((_target >> 32) & 255)
                        AND ((_guess >> 32) & 255) <> ((_target >> 32) & 255)
                        )::INT
                >=
                (((_guess >> (8 * _position)) & 255) = ((_guess >> 0) & 255)
                        AND ((_guess >> 0) & 255) <> ((_target >> 0) & 255)
                        )::INT +
                (_position >= 1
                        AND ((_guess >> (8 * _position)) & 255) = ((_guess >> 8) & 255)
                        AND ((_guess >> 8) & 255) <> ((_target >> 8) & 255)
                        )::INT +
                (_position >= 2
                        AND ((_guess >> (8 * _position)) & 255) = ((_guess >> 16) & 255)
                        AND ((_guess >> 16) & 255) <> ((_target >> 16) & 255)
                        )::INT +
                (_position >= 3
                        AND ((_guess >> (8 * _position)) & 255) = ((_guess >> 24) & 255)
                        AND ((_guess >> 24) & 255) <> ((_target >> 24) & 255)
                        )::INT +
                (_position >= 4
                        AND ((_guess >> (8 * _position)) & 255) = ((_guess >> 32) & 255)
                        AND ((_guess >> 32) & 255) <> ((_target >> 32) & 255)
                        )::INT
                )::INT
        END
$$
LANGUAGE 'sql';

CREATE OR REPLACE FUNCTION fn_match(_guess BIGINT, _target BIGINT)
RETURNS INT
IMMUTABLE PARALLEL SAFE
COST 50000
AS

$$

SELECT  fn_match(_guess, _target, 0) +
        fn_match(_guess, _target, 1) * 3 +
        fn_match(_guess, _target, 2) * 9 +
        fn_match(_guess, _target, 3) * 27 +
        fn_match(_guess, _target, 4) * 81

$$
LANGUAGE 'sql';

Commands completed successfully

Commands completed successfully

Общее время выполнения: 00:00:00.087

# Unit tests

In [6]:
SELECT  target, guess,
        CASE
        WHEN fn_match_text(fn_match(
                fn_word_to_bigint(guess),
                fn_word_to_bigint(target))
                ) = colors
        THEN
                '✅'
        ELSE    '❌'
        END
FROM    (
        VALUES
        ('AAABB', 'BBAAA', 'YYGYY'),
        ('GHOTI', 'GHOTI', 'GGGGG'),
        ('REBUS', 'RUBUS', 'GBGGG'),
        ('BANAL', 'ANNAL', 'YBGGG'),
        ('BANAL', 'UNION', 'BYBBB'),
        ('BANAL', 'ALLOY', 'YYBBB'),
        ('BANAL', 'BANAL', 'GGGGG'),
        ('ABBEY', 'ABBEY', 'GGGGG'),
        ('ABBEY', 'ABYSS', 'GGYBB'),
        ('ABBEY', 'KEBAB', 'BYGYY'),
        ('ABBEY', 'BABES', 'YYGGB'),
        ('ABBEY', 'OPENS', 'BBYBB'),
        ('DUVET', 'ADDED', 'BYBGB')
        ) AS q (target, guess, colors)




(13 row(s) affected)

Общее время выполнения: 00:00:00.197

target,guess,case
AAABB,BBAAA,✅
GHOTI,GHOTI,✅
REBUS,RUBUS,✅
BANAL,ANNAL,✅
BANAL,UNION,✅
BANAL,ALLOY,✅
BANAL,BANAL,✅
ABBEY,ABBEY,✅
ABBEY,ABYSS,✅
ABBEY,KEBAB,✅


# Single word

Let's find the worst case for the word **ALOHA**

In [17]:
SELECT  fn_match_text(fn_match(
                fn_word_to_bigint('ALOHA'),
                fn_word_to_bigint(target.word)
                )) AS colors,
        COUNT(*) matches
FROM    wordle AS target
GROUP BY
        colors
ORDER BY
        matches DESC

(110 row(s) affected)

Общее время выполнения: 00:00:02.075

colors,matches
BBBBB,2852
YBBBB,1954
BBYBB,1441
BYBBB,724
YYBBB,536
BBGBB,484
BBBYB,425
BYYBB,394
YBYBB,348
YBBYB,300


# All words

Now, let's find the "best worst case" for every word from the list.

We are using a CTE to calculate the numeric values of the words.

The function to calculate the numeric value has high cost, so the optimizer chooses to materialize the CTE.

In [5]:
WITH    numeric_wordle AS
        (
        SELECT  fn_word_to_bigint(word) AS word
        FROM    wordle
        )
SELECT  fn_bigint_to_word(word), MAX(matches) AS worst
FROM    (
        SELECT  guess.word,
                fn_match(guess.word, target.word) AS colors,
                COUNT(*) matches
        FROM    numeric_wordle AS guess
        CROSS JOIN
                numeric_wordle AS target
        GROUP BY
                guess.word, colors
        ) q
GROUP BY
        word
ORDER BY
        worst ASC
-- The smaller, the better. The optimal candidate will come out first.

(12478 row(s) affected)

Отображаются первые 5000 строк.

Общее время выполнения: 00:04:31.415

fn_bigint_to_word,worst
SERAI,659
SOARE,724
PASEO,747
AEROS,749
STOAE,781
REANS,785
NEARS,785
NARES,785
KAIES,788
REALS,791


# Limiting the pool

To play the actual game of Wordle using the Minimax algorithm, we always start with the word **SERAI**, because in the worst case it does better that all other words.

To figure out our next guess we need to run the algorithm, limiting the pool of words to those compatible with previous guesses.

We do it by putting our previous guesses and their color matches into a CTE with inline data, and running the aggregate function `BOOL_AND` in a scalar subquery.

In [15]:
WITH    guesses (guess, colors) AS
        (
        VALUES
        ('SERAI', 'BBBBB'),
        ('DONUT', 'BBBBB')
        )
SELECT  word
FROM    wordle
WHERE   (
        SELECT  BOOL_AND(
                fn_match(
                        fn_word_to_bigint(guess),
                        fn_word_to_bigint(word)
                        ) = fn_match_color(colors)
                        )
        FROM    guesses
        )

(7 row(s) affected)

Общее время выполнения: 00:00:03.614

word
FLYBY
GHYLL
GLYPH
GYPPY
LYMPH
PYGMY
XYLYL


# Playing Wordle

Let's play the actual game of Wordle!

We combine the main algorithm query with the pool-limiting query.

In [3]:
WITH    guesses (guess, colors) AS
        (
        VALUES
        ('SERAI', 'YYBBB'),
        ('LOSEN', 'BBYYB'),
        ('THESE', 'YBGGB')
        ),
        numeric_wordle AS
        (
        SELECT  fn_word_to_bigint(word) AS word
        FROM    wordle
        WHERE   (
                SELECT  BOOL_AND(
                        fn_match(
                                fn_word_to_bigint(guess),
                                fn_word_to_bigint(word)
                                ) = fn_match_color(colors)
                                )
                FROM    guesses
                )
        )
SELECT  fn_bigint_to_word(word), MAX(matches) AS worst
FROM    (
        SELECT  guess.word,
                fn_match(guess.word, target.word) AS colors,
                COUNT(*) matches
        FROM    numeric_wordle AS guess
        CROSS JOIN
                numeric_wordle AS target
        GROUP BY
                guess.word, colors
        ) q
GROUP BY
        word
ORDER BY
        worst ASC
LIMIT 1

(1 row(s) affected)

Общее время выполнения: 00:00:06.348

fn_bigint_to_word,worst
GUEST,1
